In [1]:
import gym 
import numpy as np
from custom_envs.gridworlds import WindyGridworldEnv, SimpleGridworldEnv
from custom_envs.mountain_car import MountainCarEnv

from function_approximators.function_approximators import NeuralNetwork, LinearModel, DecisionTree, RandomForest, ExtraTrees, GradientBoostingTrees, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, eGaussianProcess, OnlineGaussianProcess
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, RationalQuadratic, Matern
from sklearn.metrics.pairwise import rbf_kernel, chi2_kernel, laplacian_kernel 

from utils.train_utils import train, solve, train_time
from utils.plot_utils import plot_returns

from agents.av_agents import DQNAgent, LinearAgent, NonParametricAgent, eGaussianProccessAgent, OnlineGaussianProccessAgent

import operator


In [2]:
function_approximators = [NeuralNetwork, LinearModel, DecisionTree, RandomForest, ExtraTrees, GradientBoostingTrees, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, eGaussianProcess, OnlineGaussianProcess]
agents = [DQNAgent, LinearAgent, *[NonParametricAgent]*7, eGaussianProccessAgent, OnlineGaussianProccessAgent]

RENDER = False
env = gym.make("CartPole-v1")
# env = gym.make("Acrobot-v1")
# env = gym.make("MountainCar-v0")
# env = WindyGridworldEnv()
# env = gym.make("LunarLander-v2")
# env = SimpleGridworldEnv()
# env = MountainCarEnv()

In [6]:
# DQN Config
CONFIG_DQN = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "learning_rate": 0.00075,
    "hidden_size": (32,32),
    "target_update_freq": 200,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "plot_loss": False,
    "epsilon": 1,
    "max_deduct": 0.97,
    "decay": 0.25,
    "lr_step_size": 1000,
    "lr_gamma": 0.95,
    "max_steps": 200,
    "non_param": False,
}

# Linear Config
CONFIG_LINEAR = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "learning_rate": 0.02,
    "target_update_freq": 50,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e7),
    "plot_loss": False,
    "epsilon": 1,
    "max_steps": 200,
    "poly_degree": 2,
    "max_deduct": 0.97,
    "decay": 0.5,
    "lr_step_size": 1000,
    "lr_gamma": 0.99,
    "non_param": False,
}

# Decision Tree Config
CONFIG_DT = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"criterion":"mse","max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
    "feature_names": ["Cart Position", "Cart Velocity", "Pole Angle", "Pole Angular Velocity", "Action: Push Left", "Action: Push Right"],
    "plot_name": "dt_depth=8",
}

# Random Forest Config
CONFIG_RF = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 5,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_estimators": 10,"max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
}

# Extra Trees Config
CONFIG_ET = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 5,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_estimators": 10, "max_depth": 10, "min_samples_split": 20, "min_samples_leaf": 5},
}

# Gradient Boosting Trees Config
CONFIG_GBT = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e5),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"loss":"ls","learning_rate":0.1, "n_estimators": 10,"max_depth": 10, "min_samples_split": 10, "min_samples_leaf": 10},
}

# Support Vector Regressor Config
CONFIG_SVR = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"kernel":"rbf", "degree": 2, "C": 2},
}


# K-Neighbors Regressor Config
CONFIG_KNR = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.3,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"n_neighbors":5, "weights": "uniform", "algorithm": "auto", "leaf_size": 30},
}

# Gaussian Process Config
CONFIG_GP = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 10,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"alpha": 1e-10, "normalize_y": False, "kernel":  RBF(length_scale=0.05, length_scale_bounds="fixed")},
}

# Explorative Gaussian Process Config
CONFIG_eGP = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "model_save_freq": 2000,
    "model_save_capacity": 10,
    "update_freq": 10,
    "batch_size": 512,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"alpha": 1e-10, "normalize_y": False, "kernel":  RBF(length_scale=0.05, length_scale_bounds="fixed")},
}

# Online Gaussian Process Config
CONFIG_GP_Online = {
    "episode_length": 200,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 5,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "batch_size": 32,
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 200,
    "non_param": True,
    "model_params": {"sigma_0": 0.5, "kernel":  rbf_kernel, "epsilon_tol": 0.05, "basis_limit": 1000},
}

In [7]:
returns = []
train_returns = []
train_times = []
n_seeds=5

j=6
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    r, _, t, times = train(env, 
            CONFIG_SVR, 
            fa=function_approximators[j], 
            agent = agents[j], 
            render=RENDER,
            online=False)
    env.close()
    returns.append(r)
    train_returns.append(t)
    train_times.append(times)
    


  3%|▎         | 527/20000 [00:00<00:05, 3757.82it/s]
 Run: 1 

  5%|▌         | 1019/20000 [00:03<01:13, 258.56it/s]Evaluation at timestep 1019 returned a mean returns of 125.8
Epsilon = 0.8435666666666667
 10%|█         | 2028/20000 [00:13<02:56, 101.63it/s]Evaluation at timestep 2028 returned a mean returns of 200.0
Epsilon = 0.6846
 15%|█▌        | 3021/20000 [00:25<03:23, 83.37it/s]Evaluation at timestep 3021 returned a mean returns of 196.0
Epsilon = 0.530225
 21%|██        | 4154/20000 [00:40<03:30, 75.44it/s]Evaluation at timestep 4154 returned a mean returns of 200.0
Epsilon = 0.37395
 26%|██▌       | 5154/20000 [00:56<03:36, 68.49it/s]Evaluation at timestep 5154 returned a mean returns of 200.0
Epsilon = 0.21561666666666668
 31%|███       | 6125/20000 [01:12<03:43, 62.20it/s]Evaluation at timestep 6125 returned a mean returns of 197.6
Epsilon = 0.06187500000000001
 36%|███▌      | 7105/20000 [01:31<03:47, 56.61it/s]Evaluation at timestep 7105 returned a mean returns of 194.5


KeyboardInterrupt: 

In [5]:
with open(f'cartpole_eval_Support Vectors.csv', 'ab') as eval:
    for i in range(n_seeds):
        np.savetxt(eval, [returns[i]], delimiter=',')

In [6]:
with open(f'cartpole_train_Support Vectors.csv', 'ab') as train:
    for i in range(n_seeds):
        np.savetxt(train, [train_returns[i]], delimiter=',')
        np.savetxt(train, [train_times[i]], delimiter=',')

In [43]:
mean = [np.mean(returns, axis=0)]
std = [np.std(returns, axis=0)]
print(mean[-1])
print(std[-1])


[124.32 132.44 151.04 175.16 150.56 167.84 162.76 171.72 167.04 166.56
 176.28 172.88 169.28 172.68 174.64 187.2  181.8  179.44 183.   184.16]
[48.30927033 52.232465   61.10468394 21.82600284 57.12462166 23.30575894
 25.23121876 19.73731491 22.12741286 22.03393746 23.93210396 24.82340831
 27.35853797 24.48790722 22.81382037 12.30479581 13.88236291 16.26844799
 11.81321294 14.61144757]


In [44]:
with open(f'cartpole_Random Forest.csv', 'ab') as f:
    np.savetxt(f, mean, delimiter=',')
    np.savetxt(f, std, delimiter=',')
        

In [4]:
n_eps = []
n_steps = []
not_solved = []
n_seeds=100

j=8
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    s, e, n = solve(env, 
            CONFIG_GP, 
            fa=function_approximators[j], 
            agent = agents[j],
            target_return=195,
            op=operator.ge, 
            render=RENDER)
    env.close()
    n_eps.append(e)
    n_steps.append(s)
    not_solved.append(n)


 Run: 1 

Ep. timesteps: 200
Total timesteps: 1192
Total episodes: 49
Evaluation mean return: 200.0

 Run: 2 

Ep. timesteps: 200
Total timesteps: 2150
Total episodes: 96
Evaluation mean return: 200.0

 Run: 3 

Ep. timesteps: 200
Total timesteps: 6957
Total episodes: 179
Evaluation mean return: 200.0

 Run: 4 

Ep. timesteps: 200
Total timesteps: 2856
Total episodes: 101
Evaluation mean return: 199.6

 Run: 5 

Ep. timesteps: 200
Total timesteps: 9935
Total episodes: 200
Evaluation mean return: 200.0

 Run: 6 

Ep. timesteps: 200
Total timesteps: 960
Total episodes: 37
Evaluation mean return: 200.0

 Run: 7 

Ep. timesteps: 200
Total timesteps: 2612
Total episodes: 121
Evaluation mean return: 200.0

 Run: 8 

Ep. timesteps: 200
Total timesteps: 14668
Total episodes: 235
Evaluation mean return: 199.6

 Run: 9 

Ep. timesteps: 200
Total timesteps: 2667
Total episodes: 102
Evaluation mean return: 200.0

 Run: 10 

Ep. timesteps: 200
Total timesteps: 2069
Total episodes: 84
Evaluation me

KeyboardInterrupt: 

In [5]:
mean_eps = np.mean(n_eps)
std_eps = np.std(n_eps)
print(f"Average n_eps: {mean_eps}")
print(f"Std n_eps: {std_eps}")
print(f"St.error n_eps: {std_eps/np.sqrt(n_seeds)}")

mean_steps = np.mean(n_steps)
std_steps = np.std(n_steps)
print(f"Average n_steps: {mean_steps}0")
print(f"Std n_steps: {std_steps}")
print(f"St.error n_steps: {std_steps/np.sqrt(n_seeds)}")

print(f"Not solved: {np.sum(not_solved)} runs")

Average n_eps: 146.41176470588235
Std n_eps: 78.12965003462432
St.error n_eps: 7.812965003462432
Average n_steps: 6421.00
Std n_steps: 6276.816602917968
St.error n_steps: 627.6816602917968
Not solved: 6 runs


In [14]:
j = 10
time = train_time(env, 
        CONFIG_GP_Online, 
        fa=function_approximators[j], 
        agent = agents[j],
        online=True)
env.close()

print(time)

20029it [06:32, 51.03it/s]200.0
392.48629212379456

